**Name: Seán Kendall**

**Student Number: 20355246**

# Machine Learning Assignment 2

#### Question 1 

In [79]:
# import packages
import numpy as np 
import pandas as pd 
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

(a) Clean and prepare the dataset for machine learning analysis. You can do basic feature engineering to make your techniques scalable, but there is no need to go overboard with the dataset cleaning. Carefully consider the evaluation measure(s) that you use for this exercise and justify why you selected the particular evaluation measure(s).

In [80]:
df = pd.read_csv("census_20355246.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income-category
0,29,Local-gov,180916,Some-college,10,Separated,Adm-clerical,Unmarried,White,Female,0,0,38,United-States,<=50K
1,38,Private,159179,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1902,50,United-States,>50K
2,46,Private,128460,10th,6,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,United-States,<=50K
3,17,Private,222618,11th,7,Never-married,Sales,Own-child,Black,Female,0,0,30,United-States,<=50K
4,34,Private,117444,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,65,United-States,>50K


In [81]:
df.shape

(10000, 15)

There are 10000 rows in this dataset. The columns include 14 features and one target column, "Income-category". 

In [82]:
df.dtypes

age                  int64
 workclass          object
 fnlwgt              int64
 education          object
 education-num       int64
 marital-status     object
 occupation         object
 relationship       object
 race               object
 sex                object
 capital-gain        int64
 capital-loss        int64
 hours-per-week      int64
 native-country     object
 Income-category    object
dtype: object

There are 6 numerical columns "int" and the rest are of type "object". All are appropriate.

In [83]:
column_names = df.columns
print(column_names)

Index(['age', ' workclass', ' fnlwgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' Income-category'],
      dtype='object')


Column names need to stripped of leading space.

In [84]:
df.nunique()

age                   71
 workclass             9
 fnlwgt             8541
 education            16
 education-num        16
 marital-status        7
 occupation           15
 relationship          6
 race                  5
 sex                   2
 capital-gain        100
 capital-loss         71
 hours-per-week       81
 native-country       41
 Income-category       2
dtype: int64

In [85]:
df[" Income-category"].unique()

array([' <=50K', ' >50K'], dtype=object)

The two target values in the dataset are " <=50k" and " >50k". Could be converted to 0 and 1 using LabelEncoder.

In [86]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,10000.0,38.5154,13.688488,17.0,28.0,37.0,47.0,90.0
fnlwgt,10000.0,190348.9101,106557.798654,12285.0,117615.0,179482.5,237622.5,1268339.0
education-num,10000.0,10.0743,2.569093,1.0,9.0,10.0,12.0,16.0
capital-gain,10000.0,1006.9754,7083.766639,0.0,0.0,0.0,0.0,99999.0
capital-loss,10000.0,82.6338,393.864315,0.0,0.0,0.0,0.0,4356.0
hours-per-week,10000.0,40.3614,12.110568,1.0,40.0,40.0,45.0,99.0


Above gives a nice overview of the numerical values in the dataset. The mean and std for these values is very broad in terms of range.

In [87]:
df.isna().sum()

age                 0
 workclass          0
 fnlwgt             0
 education          0
 education-num      0
 marital-status     0
 occupation         0
 relationship       0
 race               0
 sex                0
 capital-gain       0
 capital-loss       0
 hours-per-week     0
 native-country     0
 Income-category    0
dtype: int64

There are no missing values in the dataset.

Are there duplicate records?

In [88]:
duplicate_records = df[df.duplicated()]
duplicate_records

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income-category


No duplicate records.

Inspecting categorical features:

In [89]:
categorical_features = selector(dtype_include=["object"])
categorical_features = categorical_features(df)
data_categorical = df[categorical_features]
data_categorical.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,Income-category
0,Local-gov,Some-college,Separated,Adm-clerical,Unmarried,White,Female,United-States,<=50K
1,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K
2,Private,10th,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,United-States,<=50K
3,Private,11th,Never-married,Sales,Own-child,Black,Female,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K


In [90]:
target = data_categorical.pop(" Income-category")
target.head()

0     <=50K
1      >50K
2     <=50K
3     <=50K
4      >50K
Name:  Income-category, dtype: object

In [91]:
target.value_counts()

 <=50K    7600
 >50K     2400
Name:  Income-category, dtype: int64

Above you can see there is not an equal balance of values in the target column. " <=50K" is the dominant class in this dataset and is almost around three times larger than " >50K". This should be taken into consideration when choosing an evaluation measure.  

In [92]:
data_categorical.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,Local-gov,Some-college,Separated,Adm-clerical,Unmarried,White,Female,United-States
1,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,10th,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,United-States
3,Private,11th,Never-married,Sales,Own-child,Black,Female,United-States
4,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States


Education can be converted using the OrdinalEncoder. 

What to do with the numerical features...

In [93]:
df[" capital-loss"].unique()

array([   0, 1902, 1573,  419, 2246, 1741, 1408, 1887, 2179, 1485, 1825,
       1876, 2444, 1579, 2415, 1977, 1590, 2392, 1651, 1740, 2258, 1504,
       1726, 1980, 1564, 2205,  625, 2824, 1848, 1602, 2129, 1092, 1721,
       2339, 3683, 2042, 1617, 1974, 2559,  880, 2231, 2377, 1628, 1719,
       1138, 4356, 2051, 1762, 1594, 1669, 1672, 2001, 2002, 1668, 1648,
       2206, 1258, 2603, 1380, 2149, 3900,  323, 2057, 2547, 2754,  213,
       1735, 2467,  974,  653, 2457], dtype=int64)

#### Potential problems and data quality issues are:

(1) Feature names have a space before the first word.  

(2) Target column is a object datatype. 

(3) Categorical features need to be converted to numerical. 

(4) Numerical features...

#### Solutions:

(1) Strip leading space from column names.

(2) Convert target column to 0 and 1 classes. This can be done with LabelEncoder.

(3) Convert categorical features to numerical using:
- OrdinalEncoder to convert "Education" feature as it has order. 
- OneHotEncoder to convert the rest of the categorical features to numerical.

#### (1) Fixing feature names: 

In [94]:
df.columns = df.columns.str.strip()
data_categorical.columns = data_categorical.columns.str.strip()
column_names = df.columns
print(column_names)


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Income-category'],
      dtype='object')


Feature names no longer have a leading space.

#### (2) Converting target to values to 1 and 0 using LabelEncoder:

In [95]:
label_encoder = LabelEncoder()
# transform method is similar to predict for 
target_encoded = pd.DataFrame(label_encoder.fit_transform(target), columns=[" >50K"])
target_encoded.head()

,>50K
0,0
1,1
2,0
3,0
4,1


In [96]:
target_encoded.value_counts()

 >50K
0        7600
1        2400
dtype: int64

#### (3) Converting categorical features to numerical:

First using OrdinalEncoder to convert "Education" feature:

In [97]:
data_categorical["education"]

0        Some-college
1        Some-college
2                10th
3                11th
4           Bachelors
            ...      
9995             11th
9996          HS-grad
9997        Assoc-voc
9998          Masters
9999          5th-6th
Name: education, Length: 10000, dtype: object

In [99]:
ordinal_encoder = OrdinalEncoder()
education_encoded = ordinal_encoder.fit_transform(data_categorical[["education"]])
education_encoded

array([[15.],
       [15.],
       [ 0.],
       ...,
       [ 8.],
       [12.],
       [ 4.]])